In [7]:
import numpy
numpy.__file__

'/usr/local/lib/python3.10/dist-packages/numpy/__init__.py'

/venv/bin/python


In [1]:
!rm -rf diffusers
!git clone https://github.com/huggingface/diffusers
!cd diffusers && pip install -e .
!pip install -r diffusers/examples/dreambooth/requirements_sdxl.txt 
!pip install wandb


Cloning into 'diffusers'...
remote: Enumerating objects: 32503, done.
remote: Counting objects: 100% (10276/10276), done.
remote: Compressing objects: 100% (943/943), done.
remote: Total 32503 (delta 9704), reused 9499 (delta 9268), pack-reused 22227
Receiving objects: 100% (32503/32503), 20.50 MiB | 5.38 MiB/s, done.
Resolving deltas: 100% (24182/24182), done.
Obtaining file:///workspace/sd-xl-experiments/lora/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from accelerate.utils import write_basic_config
write_basic_config()


PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [5]:
MODEL_NAME="stabilityai/stable-diffusion-xl-base-1.0"
INSTANCE_DIR="../data/phil_1024"
OUTPUT_DIR="phil-lora-3-500-1e-4"

!accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path={MODEL_NAME}  \
  --pretrained_vae_model_name_or_path=madebyollin/sdxl-vae-fp16-fix \
  --instance_data_dir={INSTANCE_DIR} \
  --output_dir={OUTPUT_DIR} \
  --mixed_precision="bf16" \
  --instance_prompt="a photo of sks man" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --validation_prompt="beautiful sks man in a suit official portrait, smiling, detailed, 8K" \
  --validation_epochs=8 \
  --seed="0"  \
  --push_to_hub \
  --num_validation_images=2


07/28/2023 20:07:15 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'variance_type', 'dynamic_thresholding_ratio', 'thresholding', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
wandb: Currently logged in as: philippe-henri-saade (kollai). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.7
wandb: Run data is saved locally in /workspace/sd-xl-experiments/lora/wandb/run-20230728_200727-ggrlfthz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sage-river-6
wandb: ⭐️ View project at h

In [3]:
!pip install --upgrade huggingface_hub

  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:00a 0:00:01
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.13.4
    Uninstalling huggingface-hub-0.13.4:
      Successfully uninstalled huggingface-hub-0.13.4


In [5]:
1+1

2

In [4]:
import huggingface_hub

ModuleNotFoundError: No module named 'huggingface_hub'

In [1]:
from huggingface_hub.repocard import RepoCard
from diffusers import DiffusionPipeline
import torch

lora_model_id = "phil-lora-2"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(lora_model_id)
image = pipe("A picture of a sks man in a suit", num_inference_steps=25).images[0]
image.save("sks_dog.png")

ModuleNotFoundError: No module named 'huggingface_hub'

In [10]:
from huggingface_hub import notebook_login


ModuleNotFoundError: No module named 'huggingface_hub'